## Классификация текста с использованием CNN
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1U3vnZeD8aiDg5Gh-SjnEyJyfrTHSRTkB)

Используя данные отзывов IMDB, построим CNN для классификации документов на позитивный и негативный классы.

Источник изложения: https://github.com/bentrevett/pytorch-sentiment-analysis

В предположении, что PyTorch уже установлен, поставим дополнительные модули и загрузим модель для токенизации:

In [0]:
# !pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

In [0]:
# если всё-таки надо поставить torch
# !pip install torch

In [1]:
!pip install torchvision

In [2]:
!pip install torchtext

In [3]:
# Поставим spacy для предобработки данных
!pip install spacy

In [4]:
# загрузим данные для spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
# import spacy
#import en
# en_nlp = spacy.load('en_core_web_sm')

## Данные

### Подготовка
Мы будем использовать объекты класса `Field`. Они определяют, как данные будут храниться и обрабатываться.

В поле `TEXT` задаём `tokenize='spacy'`. Это определяет, что тексты будут токенизироваться с помошью [spaCy](https://spacy.io) tokenizer. Если не подать аргументов, тексты будут разбиваться по пробелам.

`LABEL` is defined by a `LabelField`, a special subset of the `Field` class specifically used for handling labels. We will explain the `dtype` argument later.

Больше про класс `Field` [здесь](https://github.com/pytorch/text/blob/master/torchtext/data/field.py).

Загрузим датасет и получим из него выборку:

In [7]:
import torch
from torchtext import data

print(torch.__version__)

SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

1.4.0


### Загрузка

В pytorch (`torchtext.datasets`) хранятся некоторые стандартные датасеты. А ещё в них встроено разделение на train и test.

In [0]:
from torchtext import datasets

In [0]:
train_src, test = datasets.IMDB.splits(TEXT, LABEL)

In [0]:
import random
train, valid = train_src.split(random_state=random.seed(SEED))

Посмотрим сколько примеров в каждой части:

In [23]:
print(f'Number of training examples: {len(train)}')
print(f'Number of testing examples: {len(test)}')

Number of training examples: 17500
Number of testing examples: 25000


Посмотрим на один пример:

In [24]:
print(vars(train.examples[0]))

{'text': ['That', "'s", 'not', 'the', 'sound', 'of', 'bees', ',', 'that', "'s", 'the', 'effect', 'induced', 'by', 'watching', 'this', 'extremely', 'long', ',', 'extremely', 'boring', ',', 'badly', 'acted', 'movie', '.', 'How', 'I', 'ever', 'made', 'it', 'through', 'all', '3', '1/2', 'hours', 'without', 'falling', 'asleep', 'I', "'ll", 'never', 'know', '.', 'The', 'plot', 'is', 'simple', '...', '3', 'thoroughly', 'unlikable', 'morons', 'talk', 'about', 'sex', 'for', '3', '1/2', 'hours', '.', 'And', 'you', 'thought', 'Rohmer', 'was', 'deadly', '.', 'This', 'is', 'even', 'worse', ',', 'if', 'that', "'s", 'possible', '.', '>', 'I', 'must', 'really', 'be', 'a', 'masochist', 'if', 'I', 'could', 'watch', 'this', 'entire', 'movie', 'without', 'turning', 'it', 'off', '...', 'or', 'killing', 'someone', '.'], 'label': 'neg'}


In [25]:
train.examples[0].text[:10]

['That', "'s", 'not', 'the', 'sound', 'of', 'bees', ',', 'that', "'s"]

In [26]:
train.examples[0].label

'neg'

### Обработка

Теперь построим _vocabulary_. Это, по большому счёту, таблица, в которой каждое слово соответствует индексу. Индекс используется для создания _one-hot_ векторов.

![](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/9210842371c3bbde7b2007051dafa4c74d9768cd/assets/sentiment5.png)

Количество уникальных слов 100,000 -- это очень много для векторов. Столько данных может не влезть в GPU. Поэтому мы возьмём 25,000 самых частых слов (параметр `max_size`). Если нам встретится слово, которого среди них нет, оно заменится на  `<unk>`. Например, если в предожении "This film is great and I love it" не войдёт слово "love", получится "This film is great and I `<unk>` it".

Построим словарь и загрузим предобученные эмбеддинги:


In [0]:
TEXT.build_vocab(train, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(train)

Немного посмотрим на данные.

In [27]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


25002, а не 25000 -- из-за `<unk>` и `<pad>` токенов.

В батче должны быть однаковые длины предложений, если какое-то короче -- добавляем паддинг.

![](https://raw.githubusercontent.com/bentrevett/pytorch-sentiment-analysis/9210842371c3bbde7b2007051dafa4c74d9768cd/assets/sentiment6.png)

In [28]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203700), (',', 192882), ('.', 166026), ('a', 109745), ('and', 109440), ('of', 101493), ('to', 93823), ('is', 76399), ('in', 61756), ('I', 54333), ('it', 53184), ('that', 49452), ('"', 44122), ("'s", 43191), ('this', 42429), ('-', 37208), ('/><br', 35781), ('was', 35182), ('as', 30678), ('with', 30158)]


Данные в словаре: `stoi` (**s**tring **to** **i**nt) и `itos` (**i**nt **to**  **s**tring).

In [29]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


In [30]:
print(TEXT.vocab.stoi['and'])

6


Следующий шаг подготовки данных -- создать итераторы по батчам.

Мы используем `BucketIterator`, который будет минимизировать добавлемые токены `<pad>`. Мы также можем перевести данные на GPU с помощью `torch.device`.

In [0]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size=BATCH_SIZE, 
    sort_key=lambda x: len(x.text), # сорируем тексты по длине, чтобы рядом оказывались предложения с одинаковой длиной и добавлялось меньше паддинга
    repeat=False)

## Обучение

### Вспомогательные функции

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [0]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Зададим класс нейросети

Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [0]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conved_0 = F.relu(self.conv_0(embedded).squeeze(3))
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

Сейчас мы можем использовать только три различных фильтра, хотелось бы больше. Вообще, можно воспользоваться `nn.ModuleList`, чтобы создать слои списком и сделать так, чтобы фильтров создавалось по количеству элементов в filter_sizes. [(Как здесь).](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

Смотрим на предобученные эмбеддинги:

In [21]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.5441, -0.1638, -0.3046,  ...,  0.0627,  0.6423,  1.0284],
        [-0.2572,  0.4990,  0.7320,  ..., -0.6175, -0.5058,  0.4684],
        [ 0.2858,  0.4264,  0.2140,  ...,  0.2665,  0.2573,  0.4784]])

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [0]:
import torch.optim as optim

In [0]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.cuda()

In [33]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.504, Train Acc: 73.76%, Val. Loss: 0.342, Val. Acc: 85.12%
Epoch: 02, Train Loss: 0.311, Train Acc: 87.18%, Val. Loss: 0.287, Val. Acc: 88.07%
Epoch: 03, Train Loss: 0.217, Train Acc: 91.64%, Val. Loss: 0.273, Val. Acc: 88.78%
Epoch: 04, Train Loss: 0.145, Train Acc: 94.88%, Val. Loss: 0.265, Val. Acc: 89.35%
Epoch: 05, Train Loss: 0.089, Train Acc: 97.09%, Val. Loss: 0.297, Val. Acc: 88.98%


In [34]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.315, Test Acc: 88.22%
